In [1]:
import sys
sys.path.insert(0, r'C:\Git_projects\Survey_Support')
sys.path.insert(0, r'C:\Git_projects\IPS_Legacy_Uplift')

# sys.path.append(r'C:\Git_projects\Survey_Support')
# sys.path.append(r'C:\Git_projects\IPS_Legacy_Uplift')

In [2]:
import os
import logging
import inspect
import numpy as np
import pandas as pd
from sas7bdat import SAS7BDAT
from pandas.util.testing import assert_frame_equal
from collections import OrderedDict

import survey_support
from IPSTransformation import CommonFunctions as cf

# Call JSON configuration file for error logger setup
survey_support.setup_logging('IPS_logging_config_debug.json')

# Connect to Oracle and unload parameter list
# conn = cf.get_oracle_connection()
#global parameters
# parameters = cf.unload_parameters(205)

C:\Git_projects\IPS_Legacy_Uplift\IPS_Stored_Procedures


In [3]:
# root_data_path = r"\\nsdata3\Social_Surveys_team\CASPA\IPS\Testing\Traffic_Weight"
# df_test = SAS7BDAT(root_data_path + r"\summary_merge_sum_traftot.sas7bdat" ).to_data_frame()
# df_test.columns = df_test.columns.str.upper() 
# assert_frame_equal(df_test, df_test)
# df_test['TRAFFIC_WT'] = df_test['TRAFFIC_WT'].round(3)

In [33]:
var_shiftWeight = 'shift_wt'.upper()
var_NRWeight = 'non_response_wt'.upper()
var_minWeight = 'mins_wt'.upper()
StrataDef = ['samp_port_grp_pv'.upper(),
             'arrivedepart'.upper()] 
TotalVar = 'traffictotal'.upper()
var_traffictotal = 'traffictotal'.upper()
var_serialNum = 'serial'.upper()
var_priorWeight = 'trafDesignWeight'.upper()
var_trafficWeight = 'traffic_wt'.upper()
postWeight = "postWeight".upper()

var_count = 'cases'.upper()
var_postSum = 'sum_traffic_wt'.upper()

minCountThresh = 30

GWeightVar = 'traffic_wt'.upper()

In [21]:
# get paths
root_data_path = r"\\nsdata3\Social_Surveys_team\CASPA\IPS\Testing\Traffic_Weight"    

path_to_SurveyData = root_data_path + r"\survey_input.sas7bdat"
path_to_PopTotals = root_data_path + r"\trtotals.sas7bdat" 

# get dataframes
df_survey = SAS7BDAT(path_to_SurveyData).to_data_frame()
df_survey.columns = df_survey.columns.str.upper()   

df_trtotals = SAS7BDAT(path_to_PopTotals).to_data_frame()
df_trtotals.columns = df_trtotals.columns.str.upper()  

# perform calculation
trafDesignWeight = "trafDesignWeight".upper()        
df_survey[trafDesignWeight] = df_survey[var_shiftWeight] * df_survey[var_NRWeight] * df_survey[var_minWeight]

# test code start
root_data_path = r"\\nsdata3\Social_Surveys_team\CASPA\IPS\Testing\Traffic_Weight"
df_test = SAS7BDAT(root_data_path + r"\in_1.sas7bdat").to_data_frame()    
df_test.columns = df_test.columns.str.upper() 
assert_frame_equal(df_survey, df_test, check_column_type=False) 
# test code end

In [22]:
# Summarise the population totals over the strata
df_PopTotals = df_trtotals.sort_values(StrataDef)

# Re-index the data frame
df_PopTotals.index = range(df_PopTotals.shape[0])
   
# test code start
df_trtotals_stratadef_sort = SAS7BDAT(root_data_path + r"\trtotals_stratadef_sort.sas7bdat" ).to_data_frame()
df_trtotals_stratadef_sort.columns = df_trtotals_stratadef_sort.columns.str.upper() 
assert_frame_equal(df_PopTotals, df_trtotals_stratadef_sort)
# test code end

In [23]:
df_popTotals = df_PopTotals.groupby(StrataDef)[TotalVar]\
                    .agg([(TotalVar, 'sum')]) \
                    .reset_index()
# test code start
df_poptotals_summary_1 = SAS7BDAT(root_data_path + r"\poptotals_summary_1.sas7bdat" ).to_data_frame()
df_poptotals_summary_1.columns = df_poptotals_summary_1.columns.str.upper() 
assert_frame_equal(df_popTotals, df_poptotals_summary_1)
# test code end

#df_poptotals_summary_1
# Call the GES weighting macro        
#CalWeight = None # this is passed in by SAS, but probably should not be in future code

In [8]:
# in_1_path = root_data_path + r"\in_1.sas7bdat"    
# df_in_1 = SAS7BDAT(in_1_path).to_data_frame()
# df_in_1_renamed = df_in_1.rename(columns={trafDesignWeight: trafDesignWeight}) #rename column so that dtype is not inferred

#test1 = df_in_1.drop(columns=[trafDesignWeight])

In [9]:
#assert_frame_equal(df_survey.drop(columns=[trafDesignWeight]), df_in_1.drop(columns=[trafDesignWeight])) # these are equal
# assert_frame_equal(df_survey, df_in_1_renamed) # these are equal

#df_in_1[trafDesignWeight]
#df = df_in_1.rename(columns={trafDesignWeight: trafDesignWeight})
#df[trafDesignWeight]


In [26]:
# generate_ips_tw_summary
# df_survey = SAS7BDAT(path_to_SurveyData).to_data_frame()
# df_survey.columns = df_survey.columns.str.upper() 
# trafDesignWeight = "trafDesignWeight"#.upper()        
# df_survey[trafDesignWeight] = df_survey[var_shiftWeight] * df_survey[var_NRWeight] * df_survey[var_minWeight]

# add the traffic weight to the survey data 
# TODO: check this sort is done against SAS
# sorting not actually required in python
df_survey_sorted = df_survey.sort_values(var_serialNum)

df_output_merge_final = SAS7BDAT(root_data_path + r"\output_merge_final.sas7bdat").to_data_frame()
df_output_merge_final.columns = df_output_merge_final.columns.str.upper() 
   

In [32]:
# calculate the post weight

# /* calculate the post weight */
# data _summary;
# merge &Survey (keep = &var_serialNum &var_priorWeight &StrataDef) 
# 	  &Output (keep = &var_serialNum &var_trafficWeight);
# by &var_serialNum;

# if &var_priorWeight > 0;

# postWeight = &var_priorWeight * &var_trafficWeight;

# run;

# TODO: simplify the pandas code 

# drop duplicate column (with 'None' values) across both tables before merging
df_survey_sorted_dropped = df_survey_sorted.drop(var_trafficWeight, 1)

# merge tables
df_summary_tmp = df_survey_sorted_dropped.merge(df_output_merge_final, on = var_serialNum, how = 'outer')

# only keep rows where var_priorWeight > 0
df_summary_tmp = df_summary_tmp[df_summary_tmp[var_priorWeight]>0]

# calculate and add the post weight column
df_summary_tmp[postWeight] = df_summary_tmp[var_priorWeight] * df_summary_tmp[var_trafficWeight]

keep_list = [var_serialNum, StrataDef[1], 
                 StrataDef[0], var_priorWeight, 
                 var_trafficWeight, postWeight] 

# only keep the selected columns
df_summary = df_summary_tmp[keep_list]

# # test code start - either change pandas column types or for the test just ignore the column type - prefer the latter
df_test = SAS7BDAT(root_data_path + r"\_summary.sas7bdat" ).to_data_frame()
df_test.columns = df_test.columns.str.upper() 
assert_frame_equal(df_summary, df_test, check_column_type=False)
# # test code end
# df_test.columns
#keep_list

In [31]:
keep_list = [var_serialNum, StrataDef[1], 
                 StrataDef[0], var_priorWeight, 
                 var_trafficWeight, postWeight] 
keep_list

['SERIAL',
 'ARRIVEDEPART',
 'SAMP_PORT_GRP_PV',
 'TRAFDESIGNWEIGHT',
 'TRAFFIC_WT',
 'POSTWEIGHT']

In [12]:
# Summarise the results by strata
df_summary_sorted = df_summary.sort_values(StrataDef)

# Re-index the data frame
df_summary_sorted.index = range(df_summary_sorted.shape[0])
    
# test code start
df_test = SAS7BDAT(root_data_path + r"\_summary_stratadef_sort.sas7bdat" ).to_data_frame()
df_test.columns = df_test.columns.str.upper() 
assert_frame_equal(df_summary_sorted, df_test, check_column_type=False)
# test code end
#df_summary_sorted

In [13]:
# proc summary data = _summary;
# var postWeight;
# by &StrataDef;
# output out = &Summary (drop = _TYPE_ _FREQ_) N = &var_count sum = &var_postSum
# 	   mean(&var_trafficWeight) = &var_trafficWeight;
# run;

# keep this for reference as the methods are not deprecated, but much more code
# df_tmp_1 = df_summary_sorted.groupby(StrataDef)[postWeight]\
#                                             .agg([(var_count, 'count'), 
#                                                   (var_postSum, 'sum')])\
#                                             .reset_index()

# df_tmp_2 = df_summary_sorted.groupby(StrataDef)[var_trafficWeight]\
#                                             .agg([(var_trafficWeight, 'mean')])\
#                                             .reset_index()

        
# df_summary_varpostweight = df_summary_sorted.groupby(StrataDef)\
#                                             .agg({
#                                                 postWeight: {'count'},
#                                                 var_trafficWeight: 'mean'})\
#                                             .rename(columns={postWeight: var_count, var_trafficWeight: var_trafficWeight})\
#                                             .reset_index()

# method will possibly be deprecated - may not be an issue
df_tmp5 = df_summary_sorted.groupby(StrataDef)\
   .agg({              
       postWeight: {var_count: 'count', var_postSum: 'sum'}, 
       var_trafficWeight: {var_trafficWeight: 'mean'},
   })

# drop the additional column indexes
df_tmp5.columns = df_tmp5.columns.droplevel()

# reset indexes to keep them aligned
df_tmp5 = df_tmp5.reset_index()

# reorder columns for SAS comparison
col_order = [StrataDef[0],StrataDef[1], var_count, var_postSum, var_trafficWeight]
df_summary_varpostweight = df_tmp5[col_order]

# test code start
df_test = SAS7BDAT(root_data_path + r"\summary_varpostweight.sas7bdat" ).to_data_frame()
df_test.columns = df_test.columns.str.upper() 
assert_frame_equal(df_summary_varpostweight, df_test, check_column_type=False, check_dtype=False)
# test code end

c:\python27\lib\site-packages\pandas\core\groupby.py:4189: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [15]:
# add in the traffic totals
df_popTotals_stratadef_sort= df_popTotals.sort_values(StrataDef)

# Re-index the data frame
df_popTotals_stratadef_sort.index = range(df_popTotals_stratadef_sort.shape[0])

# test code start
df_test = SAS7BDAT(root_data_path + r"\popTotals_stratadef_sort.sas7bdat" ).to_data_frame()
df_test.columns = df_test.columns.str.upper() 
assert_frame_equal(df_popTotals_stratadef_sort, df_test, check_column_type=False)
# test code end

In [16]:
# data &Summary;
# retain &StrataDef &var_count &var_trafficTotal &var_postSum &var_trafficWeight;
# merge &Summary &TrafficTotals (keep = &StrataDef &var_trafficTotal);
# by &StrataDef;

# &var_trafficWeight = round(&var_trafficWeight, 0.001);
# &var_postSum = round(&var_postSum, 0.001);

# run;
#df_merged = df_summary_varpostweight.merge(df_popTotals_stratadef_sort, left_on=StrataDef)

df_merged = pd.merge(df_popTotals_stratadef_sort, df_summary_varpostweight, on=StrataDef, how='outer')

#df_summary_varpostweight
#df_popTotals_stratadef_sort
df_merged[var_trafficWeight] = df_merged[var_trafficWeight].apply(lambda x: round(x,3))    
df_merged[var_postSum ] = df_merged[var_postSum ].apply(lambda x: round(x,3))    

# # reorder columns for SAS comparison
col_order = [StrataDef[0],StrataDef[1], var_count, var_trafficTotal, var_postSum, var_trafficWeight]
df_summary_merge_sum_traftot = df_merged[col_order]
#df_summary_merge_sum_traftot

# test code start
df_test = SAS7BDAT(root_data_path + r"\summary_merge_sum_traftot.sas7bdat" ).to_data_frame()
df_test.columns = df_test.columns.str.upper() 
assert_frame_equal(df_summary_merge_sum_traftot, df_test, check_column_type=False)
# test code end

In [17]:
# Round the weights to 3dp    
df_output_merge_final[GWeightVar] = df_output_merge_final[GWeightVar].apply(lambda x: round(x,3))

# test start
df_test = SAS7BDAT(root_data_path + r"\output_rounded.sas7bdat" ).to_data_frame()
df_test.columns = df_test.columns.str.upper() 
assert_frame_equal(df_output_merge_final, df_test)
# test end
    

In [18]:
# /* append the weights to the output table */
# proc append base=&OutputData data=output force;
# run;
OutputData = 'sas_traffic_wt'.upper()
cf.insert_into_table_many(OutputData, df_output_merge_final)

# to test we would need to read the table from Oracle and then test it against SAS output


Getting Connection
INSERT into SAS_TRAFFIC_WT(SERIAL, TRAFFIC_WT) VALUES (:p0, :p1)
Records added to SAS_TRAFFIC_WT table - 19709


19709

In [19]:
# /* append summary information to summary table */
# proc append base=&summaryData data=summary force;
# run;

SummaryData = 'sas_ps_traffic'.upper()
cf.insert_into_table_many(SummaryData, df_summary_merge_sum_traftot)

# to test we would need to read the table from Oracle and then test it against SAS output

Getting Connection
INSERT into SAS_PS_TRAFFIC(SAMP_PORT_GRP_PV, ARRIVEDEPART, CASES, TRAFFICTOTAL, SUM_TRAFFIC_WT, TRAFFIC_WT) VALUES (:p0, :p1, :p2, :p3, :p4, :p5)
Records added to SAS_PS_TRAFFIC table - 82


82

In [172]:
df = pd.DataFrame({'A': [1, 1, 1, 2, 2],
                   'B': range(5),
                   'C': range(5)})

df.groupby('A')\
   .agg({'B': 'sum', 'C': 'min'})\
   .rename(columns={'B': 'foo', 'C': 'bar'})

df

,A,B,C
0,1,0,0
1,1,1,1
2,1,2,2
3,2,3,3
4,2,4,4


In [ ]:


# test code start
df_test = SAS7BDAT(root_data_path + r"\summary_merge_sum_traftot.sas7bdat" ).to_data_frame()
df_test.columns = df_test.columns.str.upper() 
assert_frame_equal(, df_test, check_column_type=False)
# test code end

In [260]:
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'],'value': [-1 ,2, 3, 4]})

df2 = pd.DataFrame({'key': ['B', 'D', 'D', 'E'],'value': [1, 2, 3, 4]})

#df1.merge(df2, on='key', how='inner')
df1

,key,value
0,A,-1
1,B,2
2,C,3
3,D,4


In [261]:
df2

,key,value
0,B,1
1,D,2
2,D,3
3,E,4


In [264]:
m = pd.merge(df1, df2, on='key', how='left')
m

,key,value_x,value_y
0,A,-1,NaN
1,B,2,1.0
2,C,3,NaN
3,D,4,2.0
4,D,4,3.0


In [213]:
def f(row):
    if (row['value'] > 0):
        return row['value']*row['value']

#df1['new'] = df1.apply(f, axis=1)

#df1["new"] = np.where(df1["value"]>0, df1["value"] * df1["value"], 0)
# df_new = df1.apply(f, axis=1)
# df_new

df1 = df1[df1['value'] > 0]
df1
    

,key,value
1,B,2
2,C,3
3,D,4


In [115]:
mylist = [var_serialNum, StrataDef, var_priorWeight, var_trafficWeight]
mylist

def flatten(l): return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]

flat_list = flatten(mylist)
flat_list

['SERIAL',
 'SAMP_PORT_GRP_PV',
 'ARRIVEDEPART',
 'TRAFDESIGNWEIGHT',
 'TRAFFIC_WT']

In [42]:
# test code start - won't work as survey_serialNum_sort.sas7bdat overwritten later
#path_survey_serialnum_sort = root_data_path + r"\survey_serialnum_sort.sas7bdat"    
#df_survey_serialnum_sort = SAS7BDAT(path_survey_serialnum_sort).to_data_frame()
#df_survey_serialnum_sort.columns = df_survey_serialnum_sort.columns.str.upper()
#assert_frame_equal(df_survey_sorted, df_survey_serialnum_sort)
#df_in_1_renamed = df_in_1.rename(columns={trafDesignWeight: trafDesignWeight}) #rename column so that dtype is not inferred
# test code end


In [104]:
var_shiftWeight = 'shift_wt'.upper()
var_NRWeight = 'non_response_wt'.upper()
var_minWeight = 'mins_wt'.upper()

root_data_path = r"\\nsdata3\Social_Surveys_team\CASPA\IPS\Testing\Traffic_Weight"    
path_to_SurveyData = root_data_path + r"\survey_input.sas7bdat"

df_survey = pd.read_sas(path_to_SurveyData)  

# Generate the design weights    
trafDesignWeight = "trafDesignWeight"#.upper()        
#df_survey[trafDesignWeight] = df_survey[var_shiftWeight] * df_survey[var_NRWeight] * df_survey[var_minWeight]   

# test code - start
#root_data_path = r"\\nsdata3\Social_Surveys_team\CASPA\IPS\Testing\Traffic_Weight"    
#in_1_path = root_data_path + r"\in_1.sas7bdat"    
#df_in_1 = pd.read_sas(in_1_path)

# assert_frame_equal(df_survey, sas_in_1)
    
#df_survey[trafDesignWeight]

#df_test = df_survey#.drop(columns=['SERIAL'])

#assert_frame_equal(df_survey, df_test)

df_survey

MemoryError: 

In [52]:
def calculate(SurveyData, OutputData, SummaryData, ResponseTable, var_serialNum,
              var_shiftWeight, var_NRWeight, var_minWeight, StrataDef, PopTotals,
              TotalVar, MaxRuleLength, ModelGroup, GWeightVar,
              GESBoundType, GESUpperBound, GESLowerBound, GESMaxDiff,
              GESMaxIter, GESMaxDist, var_count, var_trafficTotal,
              var_postSum, minCountThresh):  
    """
    Author       : Nassir Mohammad
    Date         : 31 Jan 2018
    Purpose      : Calculates the IPS traffic weight using GES
    Parameters   : 
    Returns      : The calculated factor value (float), or a None value.
    Requirements : NA
    Dependencies : NA
    """

#     Load SAS files into dataframes (this data will come from Oracle eventually)
    
#     %let SurveyData = ora_data.&SurveyData;
#     %let OutputData = ora_data.&OutputData;
#     %let summaryData = ora_data.&summaryData;
#     %let PopTotals = ora_data.&PopTotals;

    root_data_path = r"\\nsdata3\Social_Surveys_team\CASPA\IPS\Testing\Traffic_Weight"    
    path_to_SurveyData = root_data_path + r"\survey_input.sas7bdat"
    path_to_PopTotals = root_data_path + r"\trtotals.sas7bdat"   
    
#     root_data_path = r"\\nsdata3\Social_Surveys_team\CASPA\IPS\Testing\Traffic_Weight"    
#     path_to_SurveyData = root_data_path + r"\step_1_surveydata_1.sas7bdat" #.. edit
#     path_to_OutputData = root_data_path + r"\surveydata_1.sas7bdat" #.. edit
#     path_to_SummaryData = root_data_path + r"\surveydata_1.sas7bdat" #.. edit
#     path_to_PopTotals = root_data_path + r"\sas_traffic_data.sas7bdat" #..eidt
    
#     /* create a SAS dataset from the survey data */
#     data survey;
#     set &SurveyData;
#     run;     
    
    # ##########################################
    # create a SAS dataset from the survey data
    # ##########################################
    
    # Import data via SAS
    # This method works for all data sets but is slower
    #df_surveydata = SAS7BDAT(path_to_SurveyData).to_data_frame() 
    
    # This method is untested with a range of data sets but is faster
    df_surveydata = pd.read_sas(path_to_SurveyData)  
    
    # Import data via SQL
    #df_surveydata = cf.get_table_values(SurveyData)    
    
    # ##########################################
    # create a SAS dataset from the population 
    # totals
    # ##########################################
    
    # Import data via SAS
    # This method works for all data sets but is slower
    #df_trtotals = SAS7BDAT(path_to_PopTotals).to_data_frame()
    
    # This method is untested with a range of data sets but is faster
    df_trtotals = pd.read_sas(path_to_PopTotals)  
    
    # These variables are passed into SAS but not required, we also pass them in for now
    summary = "summary"
    output = "output"   
    
    df_output = do_ips_trafweight_calculation(df_surveydata, summary, var_serialNum, var_shiftWeight, 
                                        var_NRWeight, var_minWeight,StrataDef, df_trtotals, 
                                        TotalVar, MaxRuleLength, ModelGroup, output, GWeightVar, 
                                        GESBoundType, GESUpperBound, GESLowerBound, GESMaxDiff,
                                        GESMaxIter, GESMaxDist, var_count, var_trafficTotal, 
                                        var_postSum, minCountThresh)
    
    return df_output
    

In [83]:
def do_ips_trafweight_calculation(df_survey, summary, var_serialNum, var_shiftWeight, var_NRWeight,
                                     var_minWeight, StrataDef, PopTotals, TotalVar,      
                                     MaxRuleLength, ModelGroup, output, GWeightVar, 
                                     GESBoundType, GESUpperBound, GESLowerBound, GESMaxDiff,
                                     GESMaxIter, GESMaxDist, var_count, var_trafficTotal, 
                                     var_postSum, minCountThresh):
    
    """
    Author       : Nassir Mohammad
    Date         : 19 Feb 2018
    Purpose      : Calculates the IPS traffic weight using GES
    Parameters   : in = the IPS survey records for the period                                
                   summary = Oracle table to hold the summary data                            
                   var_serialNum = variable holding the record serial number (UID)            
                   var_shiftWeight     = variable holding the shift weight field name             
                   var_NRWeight = variable holding the non-response weight field name        
                   var_minWeight = variable holding the minimum weight field name            
                   StrataDef = List of classificatory variables                            
                   PopTotals = Population totals file                                         
                   TotalVar = Variable that holds the population totals                    
                   MaxRuleLength = maximum length of an auxiliary rule (e.g. 512)            
                   ModelGroup = Variable that will hold the model group number                
                   output = output dataset                                                 
                   GWeightVar = Variable that will hold the traffic weights                
                   GESBoundType = GES parameter : 'G' => cal. weights bound, 'F' => final  
                                                   weights bound                            
                   GESUpperBound = GES parameter : upper bound for weights (can be null)    
                   GESLowerBound = GES parameter : lower bound for weights (can be null)    
                   ESMaxDiff = GES parameter - maximum difference (e.g. 1E-8)                
                   GESMaxIter = GES parameter - maximum number of iterations (e.g. 50)        
                   GESMaxDist = GES parameter - maximum distance (e.g. 1E-8)                
                   var_count =  Variable holding the name of the case count field            
                   var_trafficTotal = Variable holding the name of the traffic total output
                   var_postSum = Variable holding the name of the post traffic weight sum     
                   minCountThresh = The minimum cell count threshold
    Returns      : TODO
    Requirements : TODO
    Dependencies : TODO
    """
    
    # Generate the design weights    
    trafDesignWeight = "trafDesignWeight"#.upper()        
    df_survey[trafDesignWeight] = df_survey[var_shiftWeight] * df_survey[var_NRWeight] * df_survey[var_minWeight]   
    
    # test code - start
    root_data_path = r"\\nsdata3\Social_Surveys_team\CASPA\IPS\Testing\Traffic_Weight"    
    in_1 = root_data_path + r"\in_1.sas7bdat"    
    sas_in_1 = pd.read_sas(in_1)
    
    assert_frame_equal(df_survey, sas_in_1)
    #return df_survey
    
    # test code - end
    
#     # Summarise the population totals over the strata
#     PopTotals = PopTotals.sort_values(StrataDef)
    
#     df_popTotals = PopTotals.groupby(StrataDef)[TotalVar].agg({TotalVar: 'sum'}).reset_index()
#     df_popTotals.index = range(df_popTotals.shape[0]) # required?   
    
#     # Call the GES weighting macro        
#     CalWeight = None # this is passed in by SAS, but probably should not be in future code
    
#     CalWeight = do_ips_ges_weighting(df_survey, var_serialNum, trafDesignWeight, StrataDef, df_popTotals,
#                          TotalVar, MaxRuleLength, ModelGroup, output, GWeightVar, CalWeight,
#                          GESBoundType, GESUpperBound, GESLowerBound, GESMaxDiff,
#                          GESMaxIter, GESMaxDist,) 
    
#     # Generate the summary table    
#     generate_ips_tw_summary(df_survey, output, summary, StrataDef, var_count,
#                             var_serialNum, GWeightVar, trafDesignWeight,
#                             var_trafficTotal, df_popTotals, var_postSum, minCountThresh)
    
#     # Round the weights to 3dp
#     df_output =  df_output[GWeightVar] = round(GWeightVar, 3)

In [86]:
test2 = calculate(SurveyData = 'sas_survey_subsample'
              ,OutputData = 'sas_traffic_wt'
              ,SummaryData = 'sas_ps_traffic'
              ,ResponseTable = 'sas_response'
              ,var_serialNum = 'serial'.upper()
              ,var_shiftWeight = 'shift_wt'.upper()
              ,var_NRWeight = 'non_response_wt'.upper()
              ,var_minWeight = 'mins_wt'.upper()
              ,StrataDef = ['samp_port_grp_pv'.upper(),
                            'arrivedepart'.upper()]                            
              ,PopTotals = 'sas_traffic_data'
              ,TotalVar = 'traffictotal'
              ,MaxRuleLength = '512'
              ,ModelGroup = 'C_group'
              ,GWeightVar = 'traffic_wt'.upper()
              ,GESBoundType = 'G'
              ,GESUpperBound = ''
              ,GESLowerBound = '1.0'
              ,GESMaxDiff = '1E-8'
              ,GESMaxIter = '50'
              ,GESMaxDist = '1E-8'
              ,var_count = 'cases'
              ,var_trafficTotal = 'traffictotal'
              ,var_postSum = 'sum_traffic_wt'
              ,minCountThresh = '30')

#test1["trafDesignWeight"]


AssertionError: DataFrame.columns are different

Attribute "inferred_type" are different
[left]:  mixed
[right]: unicode

In [77]:
list(test1)

[u'SERIAL',
 u'AGE',
 u'AM_PM_NIGHT',
 u'ANYUNDER16',
 u'APORTLATDEG',
 u'APORTLATMIN',
 u'APORTLATSEC',
 u'APORTLATNS',
 u'APORTLONDEG',
 u'APORTLONMIN',
 u'APORTLONSEC',
 u'APORTLONEW',
 u'ARRIVEDEPART',
 u'BABYFARE',
 u'BEFAF',
 u'CHANGECODE',
 u'CHILDFARE',
 u'COUNTRYVISIT',
 u'CPORTLATDEG',
 u'CPORTLATMIN',
 u'CPORTLATSEC',
 u'CPORTLATNS',
 u'CPORTLONDEG',
 u'CPORTLONMIN',
 u'CPORTLONSEC',
 u'CPORTLONEW',
 u'INTDATE',
 u'DAYTYPE',
 u'DIRECTLEG',
 u'DVEXPEND',
 u'DVFARE',
 u'DVLINECODE',
 u'DVPACKAGE',
 u'DVPACKCOST',
 u'DVPERSONS',
 u'DVPORTCODE',
 u'EXPENDCODE',
 u'EXPENDITURE',
 u'FARE',
 u'FAREK',
 u'FLOW',
 u'HAULKEY',
 u'INTENDLOS',
 u'KIDAGE',
 u'LOSKEY',
 u'MAINCONTRA',
 u'MIGSI',
 u'INTMONTH',
 u'NATIONALITY',
 u'NATIONNAME',
 u'NIGHTS1',
 u'NIGHTS2',
 u'NIGHTS3',
 u'NIGHTS4',
 u'NIGHTS5',
 u'NIGHTS6',
 u'NIGHTS7',
 u'NIGHTS8',
 u'NUMADULTS',
 u'NUMDAYS',
 u'NUMNIGHTS',
 u'NUMPEOPLE',
 u'PACKAGEHOL',
 u'PACKAGEHOLUK',
 u'PERSONS',
 u'PORTROUTE',
 u'PACKAGE',
 u'PROUTELATDE

In [78]:
list(test2)

[u'SERIAL',
 u'AGE',
 u'AM_PM_NIGHT',
 u'ANYUNDER16',
 u'APORTLATDEG',
 u'APORTLATMIN',
 u'APORTLATSEC',
 u'APORTLATNS',
 u'APORTLONDEG',
 u'APORTLONMIN',
 u'APORTLONSEC',
 u'APORTLONEW',
 u'ARRIVEDEPART',
 u'BABYFARE',
 u'BEFAF',
 u'CHANGECODE',
 u'CHILDFARE',
 u'COUNTRYVISIT',
 u'CPORTLATDEG',
 u'CPORTLATMIN',
 u'CPORTLATSEC',
 u'CPORTLATNS',
 u'CPORTLONDEG',
 u'CPORTLONMIN',
 u'CPORTLONSEC',
 u'CPORTLONEW',
 u'INTDATE',
 u'DAYTYPE',
 u'DIRECTLEG',
 u'DVEXPEND',
 u'DVFARE',
 u'DVLINECODE',
 u'DVPACKAGE',
 u'DVPACKCOST',
 u'DVPERSONS',
 u'DVPORTCODE',
 u'EXPENDCODE',
 u'EXPENDITURE',
 u'FARE',
 u'FAREK',
 u'FLOW',
 u'HAULKEY',
 u'INTENDLOS',
 u'KIDAGE',
 u'LOSKEY',
 u'MAINCONTRA',
 u'MIGSI',
 u'INTMONTH',
 u'NATIONALITY',
 u'NATIONNAME',
 u'NIGHTS1',
 u'NIGHTS2',
 u'NIGHTS3',
 u'NIGHTS4',
 u'NIGHTS5',
 u'NIGHTS6',
 u'NIGHTS7',
 u'NIGHTS8',
 u'NUMADULTS',
 u'NUMDAYS',
 u'NUMNIGHTS',
 u'NUMPEOPLE',
 u'PACKAGEHOL',
 u'PACKAGEHOLUK',
 u'PERSONS',
 u'PORTROUTE',
 u'PACKAGE',
 u'PROUTELATDE

In [79]:
assert_frame_equal(test1, test2)

AssertionError: 

In [61]:
root_data_path = r"\\nsdata3\Social_Surveys_team\CASPA\IPS\Testing\Traffic_Weight"    
path_to_SurveyData = root_data_path + r"\step_1_surveydata_1.sas7bdat" #.. edit
path_to_OutputData = root_data_path + r"\surveydata_1.sas7bdat" #.. edit
path_to_SummaryData = root_data_path + r"\surveydata_1.sas7bdat" #.. edit
path_to_PopTotals = root_data_path + r"\sas_traffic_data.sas7bdat" #..eidt
    
#     /* create a SAS dataset from the survey data */
#     data survey;
#     set &SurveyData;
#     run;     
    
    # empty right now
    #df_surveydata = cf.get_table_values("sas_survey_subsample")
    #print(df_surveydata)
#df_survey = pd.read_sas(path_to_SurveyData)
#df_survey

In [ ]:
#     /* create a SAS dataset from the population totals */
#     data trtotals;
#     set &PopTotals;
#     run;

    df_trtotals = pd.read_sas(path_to_PopTotals)
    #df_trtotals = SAS7BDAT(path_to_PopTotals).to_data_frame() # using existing method?  
    #print(df_trtotals)

#     %do_ips_trafweight_calculation(survey, summary, &var_serialNum, &var_shiftWeight, 
#                                    &var_NRWeight, &var_minWeight, &StrataDef, trtotals, 
#                                    &TotalVar, &MaxRuleLength, &ModelGroup, output, &GWeightVar, 
#                                    &GESBoundType, &GESUpperBound, &GESLowerBound, &GESMaxDiff,
#                                    &GESMaxIter, &GESMaxDist, &var_count, &var_trafficTotal, 
#                                    &var_postSum, &minCountThresh);

    # These variables are passed into SAS but not required, we also pass them in for now
    summary = "summary"
    output = "output"
#     
    a = "samp_port_grp_pv".upper()
    b = "arrivedepart".upper()    
    StrataDef = [a, b] #"samp_port_grp_pv arrivedepart", replace a, b
    
    #print(df_survey["mins_wt".upper()])
    
#     do_ips_trafweight_calculation(df_survey, summary, "serial", "shift_wt".upper(), 
#                                     "non_response_wt".upper(), "mins_wt".upper(),StrataDef, df_trtotals, 
#                                     "traffictotal".upper(), "512".upper(), "C_group".upper(), output, "traffic_wt".upper(), 
#                                     "G", "", "1.0", "1E-8",
#                                     "50", "1E-8", "cases", "traffictotal".upper(), 
#                                     "sum_traffic_wt".upper(), "30");

#     /* append the weights to the output table */
#     proc append base=&OutputData data=output force;
#     run;

#     df_OutputData = pd.read_sas(path_to_OutputData)
#     df_OutputData.append(df_output)    

#     /* append summary information to summary table */
#     proc append base=&summaryData data=summary force;
#     run;

#     df_SummaryData = pd.read_sas(path_to_SummaryData)
#     df_SummaryData.append(df_summary)

#     /* commit the response information to the response table */
#     %commit_ips_sas_response;
